In [ ]:
import hdbscan
import pandas as pd
from bertopic import BERTopic
from sklearn.metrics.pairwise import cosine_distances
import umap
import hdbscan
from bertopic import BERTopic
from sklearn.metrics.pairwise import cosine_distances
import umap
from bertopic.representation import KeyBERTInspired

In [ ]:
df_data = pd.read_csv('All_Text_Duplicate_Tweet.csv')

In [ ]:
df_generic = pd.read_csv('Generic_OutCome.csv')

In [ ]:
df_generic.loc[df_generic.Generic_OutCome == 'generic'].Cluster_ID

In [ ]:
df_data.loc[~df_data.Cluster_ID.isin(df_generic.loc[df_generic.Generic_OutCome == 'generic'].Cluster_ID)].drop_duplicates('Cluster_ID').clean_tweet

In [ ]:
umap_model = umap.UMAP(n_components=5, n_neighbors=2, min_dist=0.3, metric="cosine")
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=2, cluster_selection_method="eom")
representation_model = KeyBERTInspired()


In [ ]:
topic_model = BERTopic(embedding_model="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
                       representation_model=representation_model,
                       hdbscan_model=hdbscan_model, 
                       umap_model=umap_model)

In [ ]:
topic_text = df_data.loc[~df_data.Cluster_ID.isin(df_generic.loc[df_generic.Generic_OutCome == 'generic'].Cluster_ID)].drop_duplicates('Cluster_ID').clean_tweet.tolist()
topics, probs = topic_model.fit_transform(topic_text)

In [ ]:
probs

In [ ]:
topic_model.get_topic_info()

In [ ]:
new_topics = topic_model.reduce_outliers(topic_text, topics)

In [ ]:
topic_text

In [ ]:
topic_model.get_topic_info()[['Topic', 'Name']]

In [ ]:
new_topics

In [ ]:
df_data.loc[~df_data.Cluster_ID.isin(df_generic.loc[df_generic.Generic_OutCome == 'generic'].Cluster_ID)].drop_duplicates('Cluster_ID')['Topic_ID'] = new_topics

In [ ]:
data = df_data.loc[~df_data.Cluster_ID.isin(df_generic.loc[df_generic.Generic_OutCome == 'generic'].Cluster_ID)].drop_duplicates('Cluster_ID')

In [ ]:
data['Topic_ID'] = new_topics

In [ ]:
data['Topic_ID_Initial'] = topics

In [ ]:
pd.merge(data, topic_model.get_topic_info()[['Topic', 'Name']].rename(columns = {'Topic':'Topic_ID_Initial'})).to_csv('topic_result.csv', index = False)